In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import keras
from keras.models import Model
from keras.models import load_model



 


Using TensorFlow backend.


In [2]:
# Load the path where the images are kept. It can be loaded any other source as well like databases
path = '/home/madhu/venvs/ImagesAttendance'
images = []
classNames = []
myList = os.listdir(path)
#get the names of the people
for names in myList:
    img = cv2.imread(f'{path}/{names}')
    images.append(img)
    classNames.append(os.path.splitext(names)[0])
print(classNames)

['HR', 'Madhu', 'Daniel Radcliffe']


In [3]:
#Encode the images using opencv
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    
    return encodeList

In [4]:
encodeListKnown = findEncodings(images)

print('Encoding Complete')
 
cap = cv2.VideoCapture(0)
 

Encoding Complete


In [ ]:
while True:
    success, img = cap.read()
    #img = captureScreen()
    
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
 
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)
 
    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
        
        matchIndex = np.argmin(faceDis)
 
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            #print(name)
            y1,x2,y2,x1 = faceLoc
            y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            #markAttendance(name)
    
    cv2.imshow('Webcam',img)
    cv2.waitKey(1)